---
title: "Workflow for Pastoral Care with ManageBac"
author: "Francois de Ryckel"
date: "2024-12-29"
categories: [ManageBac, Appscript]
editor: source
date-modified: "2024-29-12"
---


# Turning ManageBac data into actionable insights! 

Each schools I have been to over the last 20 years have a slightly different approaches when dealing with students behaviours, discipline, academic issues or social-emotional issues.  There is a large umbrella there of concerns that I am bagging into the *"pastoral care"* label. 

The workflow will mostly depends on **How** and **Where** the relevant data are recorded.  In my earlier schools, it was papers and color slips shared to the counselor that will then put it into the *famous* student file.  I have been in schools were it was stored into the Powerschool sytem and now I am in a school where all pastoral care concerns are logged into ManageBac into their behavior section.  

The feature is practical.  Educators can log the note and, as appropriate, send directly an email to the students' guardians from there.  There are also options for what are the type of issues we are dealing with and what are the next steps.  

2 mains issues arises from these records: 

* It's really hard to do any trends analysis (students, cohorts, educators, curriculum, etc.)
* There is actually no way of tracking follow up. 

# Partial solution

To solve the first issue, here is what we have done: 

## Use MB API. 

Use the ManageBac API to download all the behaviors notes every 12 hrs and store it into a Google Sheet.  

Here is a sample code that we use. You need to API token to do this (you can find it in the admin part of MB or ask IT admin)


```{javascript}
#| code-fold: true

function getBehaviorNotes() {
  const ss = SpreadsheetApp.getActiveSpreadsheet();
  const behaviorSheet = ss.getSheetByName('behavior'); // data are stored in the behavior tab
  const token = 'FIND YOUR OWN on the MB SITE'; 

  // Add column headers to 'behavior' sheet
  const headers = [
    "Note ID",
    "Student ID",
    "First Name",
    "Last Name",
    "Email",
    "Grade",
    "Incident Time",
    "Behavior Type",
    "Notes",
    "Next Step",
    "Next Step Date",
    "Author ID",
    "Reported By",
    "Homeroom Advisor",
    "Visible to Parents",
    "Visible to Students",
    "Created At",
    "Updated At"
  ];
  behaviorSheet.getRange(1, 1, 1, headers.length).setValues([headers]);

  const url = 'https://api.managebac.com/v2/behavior/notes';
  const requestHeaders = {
    'auth-token': token
  };

  let page = 1;
  let hasMore = true;

  while (hasMore) {
    const params = {
      page: page,
      per_page: 100
    };

    const urlWithParams = `${url}?page=${params.page}&per_page=${params.per_page}`;

    const options = {
      headers: requestHeaders,
      method: 'GET',
      muteHttpExceptions: true
    };

    const response = UrlFetchApp.fetch(urlWithParams, options);
    const data = JSON.parse(response.getContentText());

    // Check if there are more pages
    hasMore = data.meta.current_page < data.meta.total_pages;

    data.behavior_notes.forEach(note => {
      let notes = note.notes;
      if (notes) {
        try {
          notes = notes.replace(/<[^>]+>/g, '');
        } catch (e) {
          console.error(`Error processing notes: ${e.message}`);
          notes = "";
        }
      }

      const noteData = [
        note.id,
        note.student_id,
        note.first_name,
        note.last_name,
        note.email,
        note.grade,
        note.incident_time,
        note.behavior_type,
        notes,
        note.next_step,
        note.next_step_date,
        note.author_id,
        note.reported_by,
        note.homeroom_advisor,
        note.visible_to_parents,
        note.visible_to_students,
        note.created_at,
        note.updated_at
      ];
      behaviorSheet.appendRow(noteData);
    });

    page++;
  }
}
```



We use AppScript to pull student pastoral care information from ManageBac into Google Sheets, then visualize trends with Looker Studio. This helps us identify and support students who need it most.